
# Sistema Antifraude com Machine Learning

Este notebook é uma evolução do projeto antifraude, incorporando:
- Modelos de Machine Learning supervisionado
- Regras baseadas em comportamento e histórico
- Considerações legais e operacionais (Desafio 2)
- Boas práticas de arquitetura, estilo e segurança

O objetivo final é criar um sistema capaz de receber dados de transações e indicar se devem ser **aprovadas** ou **recusadas** com base em dados históricos e análises preditivas.



# Sistema Antifraude v3 🕵️‍♂️

Este notebook simula um sistema antifraude simples com as seguintes regras:
- Rejeita transações se o usuário tiver feito muitas transações seguidas em pouco tempo.
- Rejeita transações acima de um limite diário.
- Rejeita transações se o usuário teve um estorno (`has_cbk = 1`) anteriormente.


In [ ]:

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Exibir todos os warnings e erros no notebook
import warnings
warnings.filterwarnings('ignore')


In [ ]:

# Substituir pelo caminho correto se necessário
df = pd.read_csv("seu_arquivo.csv", parse_dates=["transaction_date"])

# Ordenar os dados por user e data
df.sort_values(by=["user_id", "transaction_date"], inplace=True)
df.reset_index(drop=True, inplace=True)


In [ ]:

def aplicar_regras_antifraude(df):
    df['is_fraud'] = False

    # Mapeamento de chargebacks anteriores
    usuarios_com_cbk = set(df[df['has_cbk'] == 1]['user_id'].unique())

    for idx, row in df.iterrows():
        user = row['user_id']
        date = row['transaction_date']
        amount = row['transaction_amount']

        # Transações recentes no mesmo dia
        ultimas_transacoes = df[(df['user_id'] == user) & 
                                (df['transaction_date'] < date) &
                                (df['transaction_date'] >= date - timedelta(minutes=10))]

        if len(ultimas_transacoes) >= 3:
            df.at[idx, 'is_fraud'] = True
            continue

        # Total no mesmo dia (rolling window 24h)
        transacoes_24h = df[(df['user_id'] == user) &
                            (df['transaction_date'] >= date - timedelta(hours=24)) &
                            (df['transaction_date'] < date)]

        total_24h = transacoes_24h['transaction_amount'].sum() + amount
        if total_24h > 1000:  # exemplo: limite de R$1000 por 24h
            df.at[idx, 'is_fraud'] = True
            continue

        # Estorno anterior
        if user in usuarios_com_cbk:
            df.at[idx, 'is_fraud'] = True
            continue

    return df


In [ ]:

df = aplicar_regras_antifraude(df)
df[['user_id', 'transaction_date', 'transaction_amount', 'has_cbk', 'is_fraud']].head(10)
